# Ray tracing in one weekend optimized with CUDA.

#### Por Raphael Costa

#### Supercomputação

#### Professor Luciano Soares

## Objetivo do projeto

Este projeto tem como objetivo otimizar um Ray Tracer utilizando a biblioteca CUDA, desenvolvida pela NVIDIA. O CUDA permite o desenvolvimento de códigos paralelos utilizando GPUs.

Este projeto é baseado no código de Peter Shirley, em seu projeto Ray Tracing in One Weekend, escrito em 2018. (Disponível em http://www.realtimerendering.com/raytracing/Ray%20Tracing%20in%20a%20Weekend.pdf, acesso em 03/06/2019). Neste artigo, Peter descreve o passo a passo de como construir o mais simples possível Ray Tracer em C++. O Ray Tracing, por sua vez, é um método de renderização de imagens tridimensionais que exige um poder computacional muito grande para ser processado em CPU. O Ray Tracing tem de ser computado em tempo real, caso contrário, o usuário será impossibilitado de ver todos os frames limpos. 

O projeto original tem como objetivo final gerar a imagem abaixo, com resolução de 1200x800 com 10 níveis de anti-aliasing. Porém, para este projeto, otimizaremos com CUDA apenas até o capítulo 5, antes de iniciar o processo de anti-aliasing.

![imagem original](img/final.png)

Portanto, nosso objetivo final é a imagem abaixo.

![cap5](img/cap5.png)

Na imagem acima, temos 2 esferas sendo geradas, a central com raio 0.5 e uma maior, predominantemente verde, com raio 100. 

# O kernel

![grid](img/grid.jpeg)

Primeiro, vamos imaginar que nossa imagem pode abstraida como uma matriz tridimensional, com tamanho MxNx3, onde M é comprimento e N é a altura da imagem, sendo 3 as informações de cor de cada pixel: o fator R(red), fator G(green) e fator B(blue). 

Desta forma, podemos imaginar o grid acima como um grid bidimensional, de comprimento e altura iguais ao da imagem. Cada thread será responsável por calcular os 3 fatores do pixel correspondente a sua posição relativa no grid. Assim, se desejamos calcular uma imagem de tamanho 200x100, teremos um total de 200x100 = 20000 threads.

```C++
__global__ void generate(int *A, int nx, int ny,
                         int size, vec3 lower_left_corner,
                         vec3 horizontal, vec3 vertical,
                         vec3 origin, hitable **world){
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int j = blockIdx.y * blockDim.y + threadIdx.y;
  float u = float(i) / float(nx);
  float v = float(j) / float(ny);
  ray r(origin, lower_left_corner + u*horizontal + v*vertical);
  vec3 color = color_ray(r, world);
  int ir = int(255.99*color.r());
  int ig = int(255.99*color.g());
  int ib = int(255.99*color.b());
  A[(i*ny + j)*3] = ir;
  A[(i*ny + j)*3 + 1] = ig;
  A[(i*ny + j)*3 + 2] = ib;
}
```

Cada thread executará o código acima, que nomeamos kernel. As linhas
```C++
int i = blockIdx.x * blockDim.x + threadIdx.x;
int j = blockIdx.y * blockDim.y + threadIdx.y;
```
São responsáveis por dizer qual o id da thread, ou seja, será responsável por localiza-la e dizer qual pixel aquela thread calculára. Ainda, para efetuarmos as transições necessárias entre CPU e GPU, criamos uma matriz A, que armazenará os pixels da imagem a ser calculada. Assim, no início do programa, criamos duas várias, a <code>gpuA</code> e a <code>cpuA</code>. A <code>gpuA</code> será responsável por armazenar os cálculos feitos pela gpu, e, após o termino da execução do kernel, copiamos todos os valores de <code>gpuA</code> para <code>cpuA</code>, já que não podemos acessar váriaveis alocadas na GPU em códigos em execução na CPU.


# Comparação de tempos

Para efeitos de comparação, iremos comparar 5 tamanhos diferentes de imagens:

 - 200x100
 - 400x200
 - 800x400
 - 1200x800
 - 2400x1600

Para a medição do tempo que cada geração de imagem, foi utilizado a função da biblioteca, mais precisamente a <code>high_resolution_clock</code> (Disponível em: https://pt.cppreference.com/w/cpp/chrono).

Para a execução código cru, foi utilizado um MacBook Pro 2017:

 - Processador: 2,9 GHz Intel Core i7 (4 cores)
 - RAM: 16 GB 2133 MHz LPDDR3
 
Enquanto que para executar a versão otimizada com GPU foi utilizada uma instância do tipo p3.8xlarge da AWS, que disponibiliza 4 GPUs v100 NVIDIA.

Tamanho   |Raw Time (s)  |GPU Time (s)
:-------: |:-------: |:-------:
200x100   | 0.12845  |2.75045
400x200   | 0.298218 |2.98735
800x400   | 1.06488  |2.85048
1200x800  | 3.12548  |2.92556
2400x1500 | 12.4757  |3.05435

![grafico](img/graf.png)

Como podemos observar nos dados da tabela e no gráfico resultante, para tamanhos pequenos, a implementação em GPU não vale a pena. Isto se deve ao fato de que o Overhead de criação e alocação de memória para executar na GPU demora mais que o próprio cálculo da imagem, ja que alocar memória e copiar váriaveis entre CPU e GPU leva um certo tempo. Ja para tamanhos maiores de imagem a implementação em GPU é muito mais eficiente, sendo o tempo levado apenas 24% do tempo em CPU.

Um outro fator importante de se verificar é de que o tempo em GPU varia muito pouco, mesmo dobrando o tamanho da imagem. Isto se deve ao fato de que o tempo para o cálculo de todos os pixels é baixissimo, ja que são todos calculados ao mesmo tempo, a implementação em GPU toma mais tempo para alocar as váriaveis necessárias para os cálculos, tempo que não pode ser otimizado, é uma limitação do próprio hardware.

# Implementações futuras

Para melhorias futuras do projeto, podemos elencar algumas observações:

- Implementar raytracing para geração da imagem original.
- Medição de tempo apenas para o cálculo, já que na implementação sequencial, os prints dos valores dos pixels são feitos logo após o cálculo, o que pode influenciar no tempo medido.

# Bibliografia

Projeto realizado com auxílio de:
- NVIDIA Dev Blogs - https://devblogs.nvidia.com/accelerated-ray-tracing-cuda/